# Plot Simfile and SimulationData

In [1]:
import sys,os

import numpy as np
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

#this works apparently only for savefig stuff
mpl.rcParams['figure.figsize']=(6.0,4.0)    #(6.0,4.0)
mpl.rcParams['font.size']=10                #10 
mpl.rcParams['savefig.dpi']=400             #72 
mpl.rcParams['figure.subplot.bottom']=.1    #.125


plt.rc('font', family='serif')
plt.rc('text', usetex=True)

#inline Shit
%matplotlib inline
%config InlineBackend.figure_format='svg'
%config InlineBackend.rc = {'figure.facecolor': 'white', 'figure.subplot.bottom': 0.125, 'figure.edgecolor': 'white', 'savefig.dpi': 400, 'figure.figsize': (12.0, 8.0), 'font.size': 10}

#GUi shit
%matplotlib tk

mpl.get_configdir()

%load_ext autoreload
%autoreload 2
# Import General Packages from me
from Tools.Parsers import *
from Tools.BoundingBox import *


### Plot all Bodies

In [2]:
#plt.close('all')
def plotBodies(simFile, timeRange, bodyRange):
    
    dynState = SimFileReader.ReadInSimFile( simFile ,timeRange,bodyRange,True)
    print(dynState);
    
    fig, (ax0,ax1) = plt.subplots(nrows = 2);
    fig.canvas.set_window_title("SimState: " + simFile);
    n = len(dynState.bodies)
    jet = plt.get_cmap('jet') 
    cNorm  = colors.Normalize(vmin=0, vmax=n)
    scalarMap = cm.ScalarMappable(norm=cNorm, cmap=jet)
    
    plotOpts={'marker':'.' }
    
    # Position
    for i,body  in dynState.bodies.items():             
        ax0.plot(dynState.t, body[0][:,0:3], color= scalarMap.to_rgba(i),**plotOpts);
        ax1.plot(dynState.t, body[0][:,3:7], color= scalarMap.to_rgba(i),**plotOpts);
    
    
    
    ax0.grid(True)
    ax0.set_title('Position');
    ax0.set_ylabel(r'\mathbf{r}_S(t)');
    ax0.set_ymargin(0.2)
    ax0.autoscale(True,'both',False)
    
    ax1.grid(True)
    ax1.set_title('Quaternion');
    ax1.set_ylabel(r'\mathbf{a}_{KI}(t)');
    ax1.set_ymargin(0.2)
    ax1.autoscale(True,'both',False)
    
    
    # Velocity
    fig2, (ax20,ax21) = plt.subplots(nrows = 2);
    fig2.canvas.set_window_title("SimState: " + simFile);
    for i,body  in dynState.bodies.items():               
        ax20.plot(dynState.t, body[1][:,0:3], color= scalarMap.to_rgba(i),**plotOpts);
        ax21.plot(dynState.t, body[1][:,3:6], color= scalarMap.to_rgba(i),**plotOpts);
    
    ax20.grid(True)
    ax20.set_title('Velocity');
    ax20.set_ylabel(r'\mathbf{v}_S(t)');
    ax20.set_xmargin(0.0)
    ax20.set_ymargin(0.1)
    
    ax21.grid(True)
    ax21.set_title('Angular Velocity');
    ax21.set_ylabel(r'\mathbf{\omega}_{KI}(t)');
    ax21.set_xmargin(0.0)
    ax21.set_ymargin(0.1)

### Plot Simulation Data

In [3]:


def plotData(simDataFile):
    # Plot SimulationData.dat
    import csv, re;
    
    SimData= SimFileReader.ReadInSimDataFile(simDataFile);
    
    # Plot stuff
    fig, axarr = plt.subplots(2,3, sharex=True)
    fig.canvas.set_window_title("SimData: " + simDataFile);
    for t in axarr.reshape(-1): # reshape to make a linear array (numpy) to iterate over!
        t.margins(0.03)
        
    #Energy
    i=0;j=0;
    axarr[i,j].plot(SimData['SimulationTime'],SimData['TotalStateEnergy'],'b.-');
    axarr[i,j].set_title('Energy')
    #axarr[i,j].set_xlabel('time [s]');
    axarr[i,j].set_ylabel('T+V [J]');
    axarr[i,j].set_xlim((SimData['SimulationTime'][0],SimData['SimulationTime'][-1]))
    
    #Iterations
    i=0;j=1;
    axarr[i,j].plot(SimData['SimulationTime'],SimData['nGlobalIterations'],'b.');
    axarr[i,j].set_title('Iterations Prox')
    #axarr[i,j].set_xlabel('time [s]');
    axarr[i,j].set_ylabel('\# iterations');
    
    #Convergence
    i=0;j=2;
    axarr[i,j].plot(SimData['SimulationTime'],SimData['Converged'],'b.');
    axarr[i,j].set_title('Prox Converged')
    #axarr[i,j].set_xlabel('time [s]');
    axarr[i,j].set_ylabel('converged [0,1]');
    
    #Contacts Number
    i=1;j=0;
    axarr[i,j].set_title('Number of Contacts')
    axarr[i,j].plot(SimData['SimulationTime'],SimData['nContacts'],'b.');
    #axarr[i,j].set_xlabel('time [s]');
    axarr[i,j].set_ylabel('\# contacts');
    
    #TimeStep Time
    i=1;j=1;
    axarr[i,j].set_title('Time-Step time')
    axarr[i,j].plot(SimData['SimulationTime'],SimData['TimeStepTime'],'b.');
    #axarr[i,j].set_xlabel('time [s]');
    axarr[i,j].set_ylabel('[s]');
    
    #MaxOverlap
    i=1;j=2;
    axarr[i,j].set_title('MaxOverlap')
    axarr[i,j].plot(SimData['SimulationTime'],SimData['MaxOverlap'],'b-');
    #axarr[i,j].set_xlabel('time [s]');
    axarr[i,j].set_ylabel('[m]');
    fig.tight_layout()
    
    
    fig, axarr = plt.subplots(1,1, sharex=True)
    axarr.plot(SimData['SimulationTime'],SimData['PercussionPoolUsage'],'b-');
    
def plotDataIteration(simDataIterationFile,dataRange=[0,1e5],steps=100, plotOpts={'color':'r','marker':'.' }):
    
    simDataIteration = SimFileReader.ReadInSimDataIterationFile(simDataIterationFile)
    
    fig =  plt.figure();
    fig.canvas.set_window_title("SimDataIteration: "  + simDataIterationFile);
    
    plt.xlabel('Iteration (without minIter)')
    plt.ylabel('Residual \\| x^{k+1} -x^k\\| - \\|x^k\\| Tol_{rel} - Tol_{abs}')
    
    #plot first residual
    for i  in range(len(simDataIteration)):
        dR = dataRange.copy();
        #print('DataRange:', dR)
        iterationDataTime = np.atleast_1d(simDataIteration[i]);
        #print(iterationDataTime, np.shape(iterationDataTime)[0])
        #clamp
        dR[0] = min(np.shape(iterationDataTime)[0]-1, max(0,dR[0]) )
        dR[1] = max(dR[0],min(np.shape(iterationDataTime)[0]-1,dR[1]))
        #print('DataRange:', dR)
        
        nth = int((dR[1]-dR[0]) / steps);
        if(nth<=1.0):
            nth = 1;
        iterationCounter = np.arange(dR[0],dR[1]+1,nth,dtype=int)
        #print(iterationCounter)
        #print("Orig. Iterations: ", np.shape(iterationDataTime))
       
            
        #iterationDataTime2 = iterationDataTime[dataRange[0]:dataRange[1]:nth]
        #print("Clamped Iterations: ", np.shape(iterationCounter))
        plt.semilogy(iterationCounter,iterationDataTime[iterationCounter],**plotOpts)
        

### Load the Simfile

In [79]:
plt.close('all')

In [13]:

scriptpath = "./"
# Add the directory containing your module to the Python path (wants absolute paths)
sys.path.append(os.path.abspath(scriptpath))

bodyRange = [0,1]
timeRange = [0,50]


folderNumber=7
folderPath = "../SimFiles/SimDataRECORDGUI_%d/" % folderNumber
#folderPath = "SimData_7500Test/Test19/"


simFile = folderPath + "SimState.sim";
simDataFile = folderPath + "SimData.dat";
simDataIterationFile = folderPath + "SimDataIteration.dat";

## SIMFILE
plotBodies(simFile,timeRange,bodyRange)


## SIMDATA
plotData(simDataFile);


## SIMDATAITERATION
# pltOpt = {'marker':'.', 'markersize': 2,'markeredgewidth': 2, 'linewidth':0.4}
#pltOpt['label']=r'\alpha=0.5, T0'
# plotDataIteration(simDataIterationFile,dataRange=[0,50000],steps=50000,plotOpts=pltOpt);
# plt.gcf().tight_layout()
# plt.gcf().canvas.draw()

DynamicsState id:139713109376808
nSimBodies: 1
minTime: 0.0
maxTime: 1.0001
body ids: [0]
Header is  ['# GlobalTime [s]', 'SimulationTime [s]', 'TimeStepTime [s]', 'CollisionTime [s]', 'InclusionTime [s]', 'AvgIterTime [s]', 'nSimBodies', 'MaxOverlap [m]', 'GPUUsed', 'nContacts', 'nGlobalIterations', 'Converged', 'IsFinite', 'TotalTimeProx [s]', 'IterTimeProx [s]', 'TotalStateEnergy [J]', 'TotalKinEnergy [J]', 'TotalRotKinEnergy [J]', 'TotalSpinNorm [Nms]', 'PrecussionPoolSize', 'PercussionPoolUsage [%]']
columnNames:  ['GlobalTime', 'SimulationTime', 'TimeStepTime', 'CollisionTime', 'InclusionTime', 'AvgIterTime', 'nSimBodies', 'MaxOverlap', 'GPUUsed', 'nContacts', 'nGlobalIterations', 'Converged', 'IsFinite', 'TotalTimeProx', 'IterTimeProx', 'TotalStateEnergy', 'TotalKinEnergy', 'TotalRotKinEnergy', 'TotalSpinNorm', 'PrecussionPoolSize', 'PercussionPoolUsage']


In [ ]:
# print(dynState.t[205])
# print("q1:", dynState.bodies[0][0][205])
# print("u1:", dynState.bodies[0][1][205])

# print("q2:", dynState.bodies[1][0][205])
# print("u2:", dynState.bodies[1][1][205])

# print("q3:", dynState.bodies[2][0][205])
# print("u3:", dynState.bodies[2][1][205])

# print("q4:", dynState.bodies[3][0][205])
# print("u4:", dynState.bodies[3][1][205])

In [2]:

#dynState = ReadInSimFile( "../SimFiles/SimDataRECORDMPI_0/SimState.sim" ,[0,0.001],[0,1],False)
dynState = ReadInSimFile( "/media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche/first/SimStateResampled.sim" ,[0,5],[0,1],False)

#dynState = ReadInSimFile( "../SimFiles/SimDataRECORDGUI_4/SimState.sim" ,[0,50],[0,50],False)
# dynState = ReadInSimFile( "../SimFilesBrutus/Test40000/SimState.sim" ,[0,2],[0,10],False)

#plotBodies(dynState)
#dynState.bodies[0][0][48,:]

Read in SimFile at: /media/zfmgpu/Data/GabrielNuetzi/SimFiles/SimFilesBrutus/Avalanche/first/SimStateResampled.sim
--> File has: 4978.180644989014 mb
--> SimFile signature: b'MBSF' found
--> SimFile version: 2
--> nSimBodies: 500000
--> nDofqObj: 7
--> nDofuObj: 6
--> Add. Bytes Type: 1
--> Add. Bytes Per Body: 4
--> nBytesHeader: 28
--> nBytesBodyState: 116
--> nBytesState: 58000008
--> nStates: 90
--> nAdditionalWrongBytes: 0
--> minTime: 0.0
--> maxTime: 0.8900000000000007
Filtering Simfile: t = [0,0.8900000000000007]
--> readNStates: 90
--> Time t: [ 0.    0.01  0.02  0.03  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11
  0.12  0.13  0.14  0.15  0.16  0.17  0.18  0.19  0.2   0.21  0.22  0.23
  0.24  0.25  0.26  0.27  0.28  0.29  0.3   0.31  0.32  0.33  0.34  0.35
  0.36  0.37  0.38  0.39  0.4   0.41  0.42  0.43  0.44  0.45  0.46  0.47
  0.48  0.49  0.5   0.51  0.52  0.53  0.54  0.55  0.56  0.57  0.58  0.59
  0.6   0.61  0.62  0.63  0.64  0.65  0.66  0.67  0.68  0.69  0.7   0.71
  0

In [6]:
bodyRange = [0,1000000]
timeRange = [0,10]

#dynState = ReadInSimFile( "../SimFiles/SimDataRECORDMPI_0/SimState.sim" ,[0,0.001],[0,1],False)
f = "/home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/Simulations/SimStateTestEnd.sim"
dynState = SimFileReader.ReadInSimFile(f ,timeRange,bodyRange,False)


Read in SimFile at: /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/Simulations/SimStateTestEnd.sim
--> File has: 125.88504409790039 mb
--> SimFile signature: b'MBSF' found
--> SimFile version: 2
--> nSimBodies: 1000000
--> nDofqObj: 7
--> nDofuObj: 6
--> Add. Bytes Type: 5
--> Add. Bytes Per Body: 20
--> nBytesHeader: 28
--> nBytesBodyState: 132
--> nBytesState: 132000008
--> nStates: 1
--> nAdditionalWrongBytes: 0
--> minTime: 0.7019999999999553
--> maxTime: 0.7019999999999553
Filtering Simfile: t = [0.7019999999999553, 0.7019999999999553]
--> readNStates: 1
--> Time t: [ 0.702]
--> BodyIds: min/max  0 999999
--> Number of BodyIds:  1000000


In [7]:
normMax = 0
for b in dynState.bodies.values():
    normMax = max( normMax, np.linalg.norm(b.u[0,0:3]) )

In [14]:
import math

math.sqrt(2*(1)/9.81)

0.4515236409857309